In [1]:
%load_ext autotime

In [2]:
import sys
#sys.path.append('/scratch/pradap/python-work/enrique/')
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

time: 1.12 ms


In [3]:
import magellan as mg
import pandas as pd


DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 2.76 s


In [4]:
mg.init_jvm()

True

time: 131 ms


In [5]:
mg.rel_diff(0, 0)

0

time: 2.7 ms


In [5]:
# Read walmart books data
wal =  mg.read_csv(mg.get_install_path()+'/datasets/books/walmart.csv',
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.24 s


In [6]:
# Read bowker books data
bwk = mg.read_csv(mg.get_install_path()+'/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')

time: 71.8 ms


In [7]:
wal.describe()

,id,numAuthors,pubYear,pubMonth,pubDay
count,100000.000000,100000.000000,99987.000000,99987.000000,99987.000000
mean,10645636.986880,0.879040,2004.376669,6.452139,2.653295
std,6475136.803713,0.326082,38.334093,3.491075,6.260899
min,189746.000000,0.000000,1.000000,1.000000,1.000000
25%,4451952.500000,1.000000,2002.000000,3.000000,1.000000
50%,11086873.000000,1.000000,2008.000000,7.000000,1.000000
75%,15910933.000000,1.000000,2010.000000,9.000000,1.000000
max,23565516.000000,1.000000,2030.000000,12.000000,31.000000


time: 100 ms


In [8]:
bwk.describe()

,id,numAuthors,pubYear,pubMonth,pubDay
count,9.999000e+03,9999.000000,9759.000000,8665.000000,4301.000000
mean,9.780136e+12,1.107611,1998.084845,4.106982,14.077191
std,6.592547e+07,0.754675,12.235594,3.055992,9.939940
min,9.780000e+12,0.000000,1899.000000,0.000000,1.000000
25%,9.780071e+12,1.000000,1991.000000,1.000000,4.000000
50%,9.780140e+12,1.000000,2001.000000,3.000000,14.000000
75%,9.780200e+12,1.000000,2009.000000,8.000000,23.000000
max,9.780230e+12,5.000000,2014.000000,9.000000,31.000000


time: 25.1 ms


In [9]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'title', 'title', 
                    ['title', 'author', 'isbn'], ['title', 'author', 'isbn'])

time: 226 ms


In [10]:
len(C)

599

time: 1.88 ms


In [11]:
D = mg.sample_table(C, 350)

time: 2.97 ms


In [12]:
D['gold'] = 0

time: 1.44 ms


In [13]:
f = (D['rtable.isbn'] == D['ltable.isbn'])

time: 1.28 ms


In [14]:
D.ix[f, 'gold'] = 1

time: 2.24 ms


In [15]:
#L = mg.read_csv('label_table_demo_books.csv', ltable=wal, rtable=bwk)

time: 1.14 ms


In [16]:
L = D

time: 806 µs


In [17]:
feature_table = mg.get_features_for_matching(wal, bwk)

time: 3.13 s


In [18]:
# select a subset of matchers
f = feature_table.ix[[3,7,18,26, 53]]

time: 1.49 ms


In [19]:
# Extract feature vectors
G = mg.extract_feat_vecs(L, feature_table=f, attrs_after='gold')

time: 530 ms


In [20]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,15361143,9780195408591,1,0.000000,0.7,0.000000,1,0
1,1,16804063,9780195408591,1,0.000000,0.7,0.102564,1,0
2,2,1854541,9780195408591,1,0.086957,0.7,0.000000,1,0
3,3,14654846,9780195408591,1,NaN,0.7,0.000000,NaN,0
4,4,3951335,9780062015273,1,0.000000,1.0,0.206897,1,0
5,5,4405361,9780062015273,1,0.000000,1.0,0.000000,1,0
6,6,11248439,9780062015273,1,NaN,1.0,0.000000,NaN,0
7,7,4286713,9780217187404,1,0.000000,0.0,0.000000,0,0
8,8,1484265,9780217187404,1,0.000000,0.0,0.000000,1,0
9,9,4364532,9780060593254,1,1.000000,0.0,1.000000,1,0


time: 20 ms


In [21]:
help(mg.impute_table)

Help on function impute_table in module magellan.utils.helperfunctions:

impute_table(table, exclude_attrs=None, missing_val='NaN', strategy='mean', axis=0, val_all_nans=0)

time: 2.1 ms


In [22]:
G = mg.impute_table(G, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'])

time: 16.4 ms


In [23]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,15361143,9780195408591,1,0.000000,0.7,0.000000,1.000000,0
1,1,16804063,9780195408591,1,0.000000,0.7,0.102564,1.000000,0
2,2,1854541,9780195408591,1,0.086957,0.7,0.000000,1.000000,0
3,3,14654846,9780195408591,1,0.190248,0.7,0.000000,0.935593,0
4,4,3951335,9780062015273,1,0.000000,1.0,0.206897,1.000000,0
5,5,4405361,9780062015273,1,0.000000,1.0,0.000000,1.000000,0
6,6,11248439,9780062015273,1,0.190248,1.0,0.000000,0.935593,0
7,7,4286713,9780217187404,1,0.000000,0.0,0.000000,0.000000,0
8,8,1484265,9780217187404,1,0.000000,0.0,0.000000,1.000000,0
9,9,4364532,9780060593254,1,1.000000,0.0,1.000000,1.000000,0


time: 30.9 ms


In [24]:
# Create ML matchers
dt = mg.DTMatcher(name='DecisionTree', random_state=0)
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF', random_state=0)
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

time: 3.41 ms


In [25]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='f1')

time: 122 ms


/Users/pradap/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/pradap/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:960: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [26]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0,0,0,0.000000,0.000000,0.00000
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0,0,0,0.000000,0.000000,0.00000
2,SVM_65609135616350346441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0,0,0,0.000000,0.000000,0.00000
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0,0,0,0.076923,0.072727,0.02993
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0,0,0,0.000000,0.000000,0.00000


time: 19.8 ms


In [27]:
train_test = mg.train_test_split(G, train_proportion=0.7, random_state=0)
train = train_test['train']
test = train_test['test']

time: 9.21 ms


In [28]:
# mg.vis_debug_dt(dt, train, test, 
#         exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#         target_attr='gold') 

time: 3.2 ms


In [29]:
# mg.vis_debug_rf(rf, train, test, 
#         exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#         target_attr='gold')

time: 954 µs


In [30]:
H = mg.extract_feat_vecs(L, feature_table=feature_table, attrs_after='gold')

time: 2.69 s


In [31]:
H = mg.impute_table(H)

time: 57.8 ms


In [32]:
H

,_id,ltable.id,rtable.id,id_id_exm,id_id_anm,id_id_lev,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev,...,lang_lang_swg,editionDescr_editionDescr_jac_qgm_3_qgm_3,editionDescr_editionDescr_cos_dlm_dc0_dlm_dc0,editionDescr_editionDescr_jac_dlm_dc0_dlm_dc0,editionDescr_editionDescr_mel,editionDescr_editionDescr_lev,editionDescr_editionDescr_nmw,editionDescr_editionDescr_sw,editionDescr_editionDescr_swg,gold
0,0,15361143,9.780195e+12,0,0,0.153846,1,1,1,1,...,1.00000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
1,1,16804063,9.780195e+12,0,0,0.307692,1,1,1,1,...,1.00000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
2,2,1854541,9.780195e+12,0,0,0.384615,1,1,1,1,...,1.00000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
3,3,14654846,9.780195e+12,0,0,0.230769,1,1,1,1,...,0.95661,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
4,4,3951335,9.780062e+12,0,0,0.076923,1,1,1,1,...,1.00000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
5,5,4405361,9.780062e+12,0,0,0.153846,1,1,1,1,...,1.00000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
6,6,11248439,9.780062e+12,0,0,0.076923,1,1,1,1,...,0.95661,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
7,7,4286713,9.780217e+12,0,0,0.230769,1,1,1,1,...,0.20000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
8,8,1484265,9.780217e+12,0,0,0.153846,1,1,1,1,...,1.00000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0
9,9,4364532,9.780061e+12,0,0,0.307692,1,1,1,1,...,1.00000,0.004464,0,0,0.261688,0.038442,0.502381,0.111688,0.217749,0


time: 221 ms


In [33]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision') 

time: 73 ms


/Users/pradap/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.000000,0.000000,0.500000,0.000000,0.000000,0.100000
2,SVM_65609135616350346441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.083333,0.018868,0.045455,0.017857,0.018868,0.036876
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


time: 21.8 ms


In [35]:
# Split G into development and test
development_test = mg.train_test_split(G, train_proportion=0.6, 
                                       random_state=0)
development = development_test['train']
test = development_test['test']

time: 6.1 ms


In [50]:
len(test)

140

time: 2.17 ms


In [36]:
# First check, how does DT performs with out any triggers
# Train using development
dt.fit(table=development, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 14.4 ms


In [37]:
# Test using test
I = dt.predict(table=test, 
              exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
              target_attr='predicted',
              append=True, inplace=False)

time: 12 ms


In [38]:

eval_summary = mg.eval_matches(I, 'gold', 'predicted')

time: 12 ms


In [39]:
eval_summary['precision']

0.0

time: 2.23 ms


In [40]:
train_validation = mg.train_test_split(development, train_proportion=0.5, random_state=10)
# mg.vis_debug_dt(dt, train_validation['train'], train_validation['test'], 
#                exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#                target_attr='gold')

time: 7.61 ms


In [41]:
pos_trigger = mg.MatchTrigger()
pos_trigger.add_cond_rule('author_author_lev(ltuple, rtuple) == 1', 
                          feature_table=feature_table)
pos_trigger.add_cond_status(True)
pos_trigger.add_action(1)


True

time: 6.39 ms


In [51]:
len(G)

350

time: 2.12 ms


In [52]:
res = mg.cv_matcher_and_trigger(dt, pos_trigger, G, 
                                exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                               target_attr = 'gold', k=3)

0%  100%
[###] | ETA[sec]: 0.000 

time: 1.6 s



Total time elapsed: 1.592 sec


In [53]:
res['cv_stats']

,Metric,Num folds,Fold 1,Fold 2,Fold 3,Mean score
0,precision,3,0.153846,0.076923,0.055556,0.095442
1,recall,3,0.666667,0.500000,1.000000,0.722222
2,f1,3,0.250000,0.133333,0.105263,0.162865


time: 35.5 ms


In [54]:
from magellan.utils.helperfunctions import print_eval_summary
print_eval_summary(res['fold_stats'][1])

Precision : 7.69% (1/13)
Recall : 50.0% (1/2)
F1 : 13.33%
False positives : 12 (out of 13 positive predictions)
False negatives : 1 (out of 104 negative predictions)
time: 1.57 ms
